<a href="https://colab.research.google.com/github/luciainnocenti/IncrementalLearning/blob/KNN-Ele/ICaRLMain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import GitHub repository

In [ ]:
import os
import logging
import sys

In [ ]:
if not os.path.isdir('./DatasetCIFAR'):
  !git clone -b KNN-Ele https://github.com/luciainnocenti/IncrementalLearning.git
  !mv 'IncrementalLearning' 'DatasetCIFAR'

Cloning into 'IncrementalLearning'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 1718 (delta 84), reused 0 (delta 0), pack-reused 1574
Receiving objects: 100% (1718/1718), 2.20 MiB | 2.38 MiB/s, done.
Resolving deltas: 100% (1092/1092), done.


# Import packages

In [ ]:
from DatasetCIFAR.data_set import Dataset 
from DatasetCIFAR.data_set import Subset
from DatasetCIFAR import ResNet
from DatasetCIFAR import utils
from DatasetCIFAR import params
from DatasetCIFAR import ICaRLModel
from torchvision import models
import torch.nn as nn
import torch
import torch.optim as optim
import torchvision
import numpy as np
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

from torchvision import transforms
from torch.utils.data import DataLoader
from torch.nn import functional as F
import random
random.seed(params.SEED)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
print(params.SEED)
print(params.NUM_WORKERS)

653
4


# Define Datasets

In [ ]:
train_transformer = transforms.Compose([transforms.RandomCrop(size = 32, padding=4),
                                         transforms.RandomHorizontalFlip(),
                                         transforms.ToTensor(),
                                         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transformer = transforms.Compose([transforms.ToTensor(),
                                         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
trainDS = Dataset(train=True)
testDS = Dataset(train=False)

Extracting data/cifar-100-python.tar.gz to data
Files already downloaded and verified


In [ ]:
train_splits = trainDS.splits
test_splits = testDS.splits

# Define Network

In [ ]:
ICaRL = ResNet.resnet32(num_classes=100)
ICaRL =  ICaRL.to(params.DEVICE)

In [ ]:
exemplars = [None]*100

test_indexes =  []
accs = []

In [10]:
for task in range(0, 100, params.TASK_SIZE):
  train_indexes = trainDS.__getIndexesGroups__(task)
  test_indexes = test_indexes + testDS.__getIndexesGroups__(task)

  train_dataset = Subset(trainDS, train_indexes, transform = train_transformer)
  test_dataset = Subset(testDS, test_indexes, transform = test_transformer)

  train_loader = DataLoader( train_dataset, num_workers=params.NUM_WORKERS, batch_size=params.BATCH_SIZE, shuffle=True)
  test_loader = DataLoader( test_dataset, num_workers=params.NUM_WORKERS, batch_size=params.BATCH_SIZE , shuffle=True )
  
  ICaRL, exemplars = ICaRLModel.incrementalTrain(task, trainDS, ICaRL, exemplars, train_transformer)

  col = []
  for i,x in enumerate( train_splits[ :int(task/10) + 1]) : 
    v = np.array(x)
    col = np.concatenate( (col,v), axis = None)
    col = col.astype(int)
  mean = None
  total = 0.0
  running_corrects = 0.0
  for img, lbl, _ in train_loader:
      img = img.float().to(params.DEVICE)
      preds, mean = ICaRLModel.classify(img, exemplars, ICaRL, task, trainDS, mean)
      preds = preds.to(params.DEVICE)
      #print("this is preds")
      #print(preds)
      #labels = utils.mapFunction(lbl, col).to(params.DEVICE)
      #print("this is lbl")
      #print(lbl)
      total += len(lbl)
      running_corrects += torch.sum(preds == lbl.data.to(params.DEVICE)).data.item()

  accuracy = float(running_corrects/total)
  print(f'task: {task}', f'train accuracy = {accuracy}')
  accs.append(accuracy)

  total = 0.0
  running_corrects = 0.0
  tot_preds = []
  tot_lab = []
  for img, lbl, _ in test_loader:
      img = img.float().to(params.DEVICE)
      preds, _ = ICaRLModel.classify(img, exemplars, ICaRL, task, trainDS, mean)
      preds = preds.to(params.DEVICE)
      #labels = utils.mapFunction(lbl, col).to(params.DEVICE)

      tot_preds = np.concatenate( ( tot_preds, preds.data.cpu().numpy() ) )
      tot_lab = np.concatenate( (tot_lab, lbl.data.cpu().numpy()  ) )#labels

      total += len(lbl)
      running_corrects += torch.sum(preds == lbl.data.to(params.DEVICE)).data.item()

  accuracy = float(running_corrects/total)
  print(f'task: {task}', f'test accuracy = {accuracy}')
 # cf = confusion_matrix(tot_lab, tot_preds)
 # df_cm = pd.DataFrame(cf, range(task + params.TASK_SIZE), range(task + params.TASK_SIZE))
 # sn.set(font_scale = .5) # for label size
 # sn.heatmap(df_cm, annot=False)
 # plt.show()

col =  [94 63 74 21 35 56 91 96 87 48]
col[:10] [94 63 74 21 35 56 91 96 87 48]
At step  0  and at epoch =  0  the loss is =  0.029880236834287643  and accuracy is =  0.1534
At step  0  and at epoch =  1  the loss is =  0.024395357817411423  and accuracy is =  0.3636
this is X_train
[[ 0.40141425 -0.71482579 -1.56871788 ... -0.5525189  -0.96622589
  -1.25172303]
 [ 0.78500778 -0.58910122 -1.92908121 ... -0.33466844 -0.8080729
  -1.29150019]
 [ 0.09285556 -0.85812695 -1.12598585 ... -0.82066733 -1.0840549
  -1.19887882]
 ...
 [-1.00729067 -0.83677245  0.29691517 ... -0.98761495 -0.1385937
  -0.73702763]
 [-0.75774822  0.08411108  2.0711545  ... -0.87322154 -0.39058897
  -0.55327971]
 [-0.89319858  0.50888036  1.23673881 ... -0.52819229 -0.49359466
  -0.13847307]]
this is y_train
[94 94 94 ... 48 48 48]
this is preds
tensor([74, 63, 87, 21, 21, 63, 56, 56, 74, 63, 56, 87, 63, 91, 56, 94, 96, 21,
        63, 96, 21, 56, 96, 96, 35, 94, 87, 21, 21, 21, 63, 74, 74, 21, 94, 94,
        87, 7

KeyboardInterrupt: ignored

In [ ]:
accs = [.897,.80,.76,.69,.64,.58,.54,.52,.48,.46]

In [ ]:
import numpy as np
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
def plotTask(pars_tasks):

  x_tasks =  np.linspace(10, 100, 10)

  plt.plot(x_tasks, pars_tasks, label=['Accuracy'])
  plt.xlabel("Epoch")
  plt.ylim(0,1)
  plt.title('Accuracy over tasks')
  plt.legend(['Accuracy'])
  plt.grid(True)
  plt.show()

In [ ]:
plotTask(accs)

In [ ]:
np.diagonal(cf)

In [ ]:
true = np.sum(cf, axis=1)
true

In [ ]:
pred = np.sum(cf, axis=0)
pred

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
pred = np.array([137,  98, 142, 112,  39,  82, 116,  96, 145, 112,  84, 110,  59,
        96,  73,  78,  50,  91,  98,  92,  64, 109,  91,  72,  38,  70,
        76,  81,  89, 102,  81, 116,  72,  59, 177,  59,  62,  63,  93,
        90,  72,  79, 102,  52, 162,  68,  67,  80,  65,  98,  80,  65,
        84,  96, 121,  53,  77, 136,  69, 103,  76,  94, 109,  79, 126,
        61, 126, 106, 182,  68, 122,  94,  90,  80,  80, 159,  98, 132,
       145,  88, 170,  77,  79,  78, 131,  74, 149,  70, 149, 114, 102,
       172, 212, 134, 124, 187, 189, 114, 126, 131])

In [ ]:
val = np.arange(100)

In [ ]:
avg = np.mean(pred)
avg

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111)
x_coordinates = val
ax.bar(x_coordinates, pred, align='center')
ax.axhline(avg, color="gray")
plt.show()

In [ ]:
i = 0
for el, name in zip(pred,trainDS.__getClassesNames__()) :
  print('classe', i, 'name', name, ',preds', el)
  i+=1

In [ ]:
train_splits

In [ ]:
trainDS.__getClassesNames__()